In [2]:
import re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import xlrd
from openpyxl import load_workbook
import os
from datetime import datetime

In [3]:
characteristics_SHOCK = ['rfid', 'subject', 'room', 'cohort', 'trial_id', 'drug', 'box',
       'start_time', 'end_time', 'start_date', 'end_date',
       'total_active_lever_presses', 'total_inactive_lever_presses',
       'total_shocks', 'total_reward', 'rewards_after_first_shock',
       'rewards_got_shock', 'reward_timestamps']

In [4]:
rfid_coc = pd.read_csv('/Users/yunyihuang/Desktop/gl_data/rfid_cocaine.csv', index_col=0)

In [5]:
# get all the sheetnames in one excel workbook
def get_sheetnames_xlsx(file_name):
    wb = load_workbook(file_name, read_only=True, keep_links=False)
    return wb.sheetnames

# clean subject id
def clean_subject_id(sid):
    sid = sid.upper()
    if 'F' in sid:
        char = 'F'
    if 'M' in sid:
        char = 'M'
        
    idx = sid.index(char)
    return sid[idx:].split('.')[0]

# convert column names into correct format
def clean_cols(s):
    if 'U' in s:
        return s.replace('U','Active ')
    elif 'V' in s:
        return s.replace('V','Inactive ')
    elif 'Y' in s:
        return s.replace('Y','Reward ')
    else:
        return s

In [6]:
input_path = '/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/shock'
output_path = '/Users/yunyihuang/Desktop/gl_data/GEORGE/COC_SHOCK'
workbooks = [os.path.join(input_path, i) for i in sorted(os.listdir(input_path)) if ('modified_final' in i)]
workbooks

['/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/shock/C01.COC.SHOCK_modified_final.xlsx',
 '/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/shock/C02.COC.SHOCK_modified_final.xlsx',
 '/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/shock/C03.COC.SHOCKDATA_modified_final.xlsx',
 '/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/shock/C04.COC.SHOCKDATA_modified_final.xlsx',
 '/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/shock/C05.COC.SHOCKDATA_modified_final.xlsx',
 '/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/shock/C08.COC.SHOCKDATA_modified_final.xlsx',
 '/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/shock/C09.COC.SHOCKDATA_modified_final.xlsx',
 '/Users/yunyihuang/George Lab Dropbox/George_Lab/Team GWAS/SA data/Cocaine/shock/C10COCAINESHOCK_modified_final.xlsx',
 '/Users/yunyihuang/George Lab Dro

In [7]:
len(workbooks)

9

In [22]:
for wb in workbooks:
    worksheets = sorted(get_sheetnames_xlsx(wb))
    print(len(worksheets))
    for ws in worksheets:
        print(ws)
        transform_old_shock(wb,ws)

3
SHOCK01_9-6-17
SHOCK02_9-11-17
SHOCK03_9-13-17
3
PRESHOCK_11-21-17
SHOCK01_11-27-17
SHOCK02_11-29-17
4
PRESHOCK_3-13-2018
SHOCK01_3-15-2018
SHOCK02_3-20-2018
SHOCK03_3-22-2018
4
PRESHOCK_5-21-2018
SHOCK01_5-23-2018
SHOCK02_5-29-2018
SHOCK03_5-31-2018
4
PRESHOCK_9-4-2018
SHOCK01_9-6-2018
SHOCK02_9-11-2018
SHOCK03_9-13-2018
1
SHOCK03_8-2-2019
2
PRESHOCK_10-29-19
SHOCK03_10-30-2019
2
PRESHOCK_2-5-2020
SHOCK03_2-6-2020
2
PRESHOCK_5-6-2020
SHOCK03_5-7-2020


In [120]:
worksheets

['PRESHOCK_3-13-2018',
 'SHOCK01_3-15-2018',
 'SHOCK02_3-20-2018',
 'SHOCK03_3-22-2018']

In [17]:
wb = workbooks[4]
worksheets = sorted(get_sheetnames_xlsx(wb))
ws = worksheets[2]
ws

'SHOCK02_9-11-2018'

In [15]:
df_existed = pd.read_csv('/Users/yunyihuang/Desktop/testDS_public_trial_shock.csv')
existed = df_existed['subject'].to_numpy()

In [18]:
transform_old_shock(wb,ws)

In [21]:
def transform_old_shock(wb,ws):
    filepath = os.path.join(input_path, wb)
    df_raw = pd.read_excel(filepath, sheet_name = ws).T.reset_index()

    # modify the header
    new_header = df_raw.iloc[0]     #grab the first row for the header
    df = df_raw[1:]                 #take the data except the header row
    df.columns = new_header 

    # get rid of 0s
    df.replace(0, np.nan, inplace=True)
    df.dropna(how='all', axis=1, inplace=True)
    df.fillna(0,inplace=True)
    
    # group timestamp columns
    colnames = df.columns.tolist()
    reward_shock_begin = colnames.index('Reward # Got Shock 1')
    reward_col_begin = colnames.index('Reward 1')
    reward_col_end = colnames.index('Rewards After First Shock')
    df['Rewards Got Shock'] = df.iloc[:,reward_shock_begin:reward_col_begin].values.tolist()
    df['Reward Timestamps'] = df.iloc[:,reward_col_begin:reward_col_end].values.tolist()
    df.drop(df.iloc[:, reward_shock_begin:reward_col_end], inplace=True, axis=1)
    
    # add extra info
    cohort = int(wb.split('/')[-1][1:3])
    raw_trial_id = ws.split('_')[0]
    
    if raw_trial_id == 'PRESHOCK':
        trial_id = raw_trial_id
    elif cohort in range(1,6):
        trial_id = raw_trial_id.replace('0','_V')
    else:
        trial_id = 'SHOCK_V3'
    
    df['room'] = ['N/A'] * len(df)
    df['cohort'] = [cohort] * len(df)
    df['trial_id'] = [trial_id] * len(df)
    df['drug'] = ['cocaine'] * len(df)
    df['end_time'] = [datetime.min.time()] * len(df)
    df['end_date'] = [datetime.min.date()] * len(df)
    
    # reorganize columns
    df.rename(columns=str.lower,inplace=True)
    df.columns = df.columns.str.replace(' ','_')
    dff = pd.merge(df, rfid_coc,  how='left', on = ['subject'])
    dff.fillna({'rfid':-999}, inplace=True)
    dff = dff[characteristics_SHOCK]
    dff['start_time'] = dff['start_time'].apply(lambda x: datetime.strptime(x, "%H:%M:%S").time())
    dff['start_date'] = dff['start_date'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y").date())
    dff.sort_values(by='subject', inplace=True)
    dff = dff[~dff['subject'].isin(existed)]

    filename = wb.split('/')[-1][:3] + '_' + ws.split('.')[0]+'_transformed.csv' 
    dff.to_csv(os.path.join(output_path, filename))

## Sketch

In [162]:
filepath = os.path.join(input_path, wb)
df_raw = pd.read_excel(filepath, sheet_name = ws).T.reset_index()

# modify the header
new_header = df_raw.iloc[0]     #grab the first row for the header
df = df_raw[1:]                 #take the data except the header row
df.columns = new_header 

In [163]:
df

,Subject,Start Date,Start Time,Experiment,Group,Box,Total Inactive Lever Presses,Total Reward,Total Active Lever Presses,Total Shocks,...,Reward 193,Reward 194,Reward 195,Reward 196,Reward 197,Reward 198,Reward 199,Reward 200,Reward 201,Rewards After First Shock
1,F501,09/04/2018,08:14:10,0,0,1,2,34,39,9,...,0,0,0,0,0,0,0,0,0,33
2,F502,09/04/2018,08:14:13,0,0,2,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,F503,09/04/2018,08:14:16,0,0,3,3,38,97,12,...,0,0,0,0,0,0,0,0,0,33
4,F504,09/04/2018,08:14:18,0,0,4,7,26,52,9,...,0,0,0,0,0,0,0,0,0,24
5,F505,09/04/2018,08:14:20,0,0,5,0,22,22,7,...,0,0,0,0,0,0,0,0,0,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Unnamed: 58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0
59,Unnamed: 59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0
60,Unnamed: 60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0
61,Unnamed: 61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [164]:
df.Subject.value_counts()

F501           1
F527           1
F521           1
F522           1
F523           1
              ..
M563           1
M564           1
M565           1
M566           1
Unnamed: 62    1
Name: Subject, Length: 62, dtype: int64

In [161]:
# get rid of 0s
df.replace(0, np.nan, inplace=True)
df.dropna(how='all', axis=1, inplace=True)
df.fillna(0,inplace=True)

df

,Subject,Start Date,Start Time,Box,Total Inactive Lever Presses,Total Reward,Total Active Lever Presses,Total Shocks,Reward # Got Shock 1,Reward # Got Shock 2,...,Reward 30,Reward 31,Reward 32,Reward 33,Reward 34,Reward 35,Reward 36,Reward 37,Reward 38,Rewards After First Shock
1,F501,09/04/2018,08:14:10,1.0,2.0,34.0,39.0,9.0,1.0,2.0,...,3232.0,3326.0,3357.0,3437.0,3520.0,0.0,0.0,0.0,0.0,33.0
2,F502,09/04/2018,08:14:13,2.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,F503,09/04/2018,08:14:16,3.0,3.0,38.0,97.0,12.0,5.0,6.0,...,2753.0,2896.0,3018.0,3096.0,3159.0,3377.0,3398.0,3457.0,3514.0,33.0
4,F504,09/04/2018,08:14:18,4.0,7.0,26.0,52.0,9.0,2.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0
5,F505,09/04/2018,08:14:20,5.0,0.0,22.0,22.0,7.0,1.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Unnamed: 58,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0
59,Unnamed: 59,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0
60,Unnamed: 60,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0
61,Unnamed: 61,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [160]:
colnames = df.columns.tolist()
reward_shock_begin = colnames.index('Reward # Got Shock 1')
reward_col_begin = colnames.index('Reward 1')
reward_col_end = colnames.index('Rewards After First Shock')
df['Rewards Got Shock'] = df.iloc[:,reward_shock_begin:reward_col_begin].values.tolist()
df['Reward Timestamps'] = df.iloc[:,reward_col_begin:reward_col_end].values.tolist()
df.drop(df.iloc[:, reward_shock_begin:reward_col_end], inplace=True, axis=1)
df

,Subject,Start Date,Start Time,Box,Total Inactive Lever Presses,Total Reward,Total Active Lever Presses,Total Shocks,Rewards After First Shock,Rewards Got Shock,Reward Timestamps
1,F501,09/04/2018,08:14:10,1.0,2.0,34.0,39.0,9.0,33.0,"[1.0, 2.0, 10.0, 12.0, 14.0, 16.0, 22.0, 24.0,...","[2.0, 33.0, 66.0, 106.0, 141.0, 195.0, 262.0, ..."
2,F502,09/04/2018,08:14:13,2.0,5.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,F503,09/04/2018,08:14:16,3.0,3.0,38.0,97.0,12.0,33.0,"[5.0, 6.0, 7.0, 15.0, 16.0, 19.0, 21.0, 23.0, ...","[41.0, 63.0, 83.0, 105.0, 169.0, 195.0, 219.0,..."
4,F504,09/04/2018,08:14:18,4.0,7.0,26.0,52.0,9.0,24.0,"[2.0, 6.0, 8.0, 13.0, 16.0, 19.0, 24.0, 25.0, ...","[60.0, 138.0, 247.0, 279.0, 305.0, 364.0, 582...."
5,F505,09/04/2018,08:14:20,5.0,0.0,22.0,22.0,7.0,21.0,"[1.0, 8.0, 10.0, 12.0, 15.0, 16.0, 22.0, 0.0, ...","[15.0, 44.0, 142.0, 353.0, 474.0, 738.0, 1033...."
...,...,...,...,...,...,...,...,...,...,...,...
58,Unnamed: 58,0,0,0.0,0.0,0.0,0.0,0.0,13.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
59,Unnamed: 59,0,0,0.0,0.0,0.0,0.0,0.0,12.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
60,Unnamed: 60,0,0,0.0,0.0,0.0,0.0,0.0,30.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
61,Unnamed: 61,0,0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [145]:
trial_id = ws.split('_')[0]
cohort = int(wb.split('/')[-1][1:3])
df['room'] = ['N/A'] * len(df)
df['cohort'] = [cohort] * len(df)
df['trial_id'] = [trial_id] * len(df)
df['drug'] = ['cocaine'] * len(df)
df['end_time'] = [datetime.min.time()] * len(df)
df['end_date'] = [datetime.min.date()] * len(df)

In [146]:
df.shape

(62, 17)

In [147]:
df.rename(columns=str.lower,inplace=True)
df.columns = df.columns.str.replace(' ','_')
df.head(10)

,subject,start_date,start_time,box,total_inactive_lever_presses,total_reward,total_active_lever_presses,total_shocks,rewards_after_first_shock,rewards_got_shock,reward_timestamps,room,cohort,trial_id,drug,end_time,end_date
1,F501,09/04/2018,08:14:10,1.0,2.0,34.0,39.0,9.0,33.0,"[1.0, 2.0, 10.0, 12.0, 14.0, 16.0, 22.0, 24.0,...","[2.0, 33.0, 66.0, 106.0, 141.0, 195.0, 262.0, ...",N/A,5,PRESHOCK,cocaine,00:00:00,0001-01-01
2,F502,09/04/2018,08:14:13,2.0,5.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",N/A,5,PRESHOCK,cocaine,00:00:00,0001-01-01
3,F503,09/04/2018,08:14:16,3.0,3.0,38.0,97.0,12.0,33.0,"[5.0, 6.0, 7.0, 15.0, 16.0, 19.0, 21.0, 23.0, ...","[41.0, 63.0, 83.0, 105.0, 169.0, 195.0, 219.0,...",N/A,5,PRESHOCK,cocaine,00:00:00,0001-01-01
4,F504,09/04/2018,08:14:18,4.0,7.0,26.0,52.0,9.0,24.0,"[2.0, 6.0, 8.0, 13.0, 16.0, 19.0, 24.0, 25.0, ...","[60.0, 138.0, 247.0, 279.0, 305.0, 364.0, 582....",N/A,5,PRESHOCK,cocaine,00:00:00,0001-01-01
5,F505,09/04/2018,08:14:20,5.0,0.0,22.0,22.0,7.0,21.0,"[1.0, 8.0, 10.0, 12.0, 15.0, 16.0, 22.0, 0.0, ...","[15.0, 44.0, 142.0, 353.0, 474.0, 738.0, 1033....",N/A,5,PRESHOCK,cocaine,00:00:00,0001-01-01
6,F506,09/04/2018,08:14:22,6.0,0.0,19.0,21.0,5.0,16.0,"[3.0, 5.0, 7.0, 12.0, 14.0, 0.0, 0.0, 0.0, 0.0...","[897.0, 931.0, 1005.0, 1058.0, 1147.0, 1349.0,...",N/A,5,PRESHOCK,cocaine,00:00:00,0001-01-01
7,M551,09/04/2018,08:14:33,9.0,10.0,2.0,3.0,1.0,0.0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[284.0, 3020.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",N/A,5,PRESHOCK,cocaine,00:00:00,0001-01-01
8,M552,09/04/2018,08:14:38,10.0,2.0,3.0,3.0,0.0,22.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[21.0, 426.0, 649.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",N/A,5,PRESHOCK,cocaine,00:00:00,0001-01-01
9,M553,09/04/2018,08:14:41,11.0,1.0,0.0,0.0,0.0,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",N/A,5,PRESHOCK,cocaine,00:00:00,0001-01-01
10,M554,09/04/2018,08:14:44,12.0,0.0,19.0,22.0,6.0,0.0,"[1.0, 4.0, 5.0, 11.0, 12.0, 19.0, 0.0, 0.0, 0....","[60.0, 101.0, 158.0, 242.0, 335.0, 371.0, 770....",N/A,5,PRESHOCK,cocaine,00:00:00,0001-01-01


In [148]:
dff = pd.merge(df, rfid_coc,  how='left', on = ['subject'])
dff

,subject,start_date,start_time,box,total_inactive_lever_presses,total_reward,total_active_lever_presses,total_shocks,rewards_after_first_shock,rewards_got_shock,reward_timestamps,room,cohort,trial_id,drug,end_time,end_date,rfid
0,F501,09/04/2018,08:14:10,1.0,2.0,34.0,39.0,9.0,33.0,"[1.0, 2.0, 10.0, 12.0, 14.0, 16.0, 22.0, 24.0,...","[2.0, 33.0, 66.0, 106.0, 141.0, 195.0, 262.0, ...",N/A,5,PRESHOCK,cocaine,00:00:00,0001-01-01,9.330001e+14
1,F502,09/04/2018,08:14:13,2.0,5.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",N/A,5,PRESHOCK,cocaine,00:00:00,0001-01-01,9.330001e+14
2,F503,09/04/2018,08:14:16,3.0,3.0,38.0,97.0,12.0,33.0,"[5.0, 6.0, 7.0, 15.0, 16.0, 19.0, 21.0, 23.0, ...","[41.0, 63.0, 83.0, 105.0, 169.0, 195.0, 219.0,...",N/A,5,PRESHOCK,cocaine,00:00:00,0001-01-01,9.330001e+14
3,F504,09/04/2018,08:14:18,4.0,7.0,26.0,52.0,9.0,24.0,"[2.0, 6.0, 8.0, 13.0, 16.0, 19.0, 24.0, 25.0, ...","[60.0, 138.0, 247.0, 279.0, 305.0, 364.0, 582....",N/A,5,PRESHOCK,cocaine,00:00:00,0001-01-01,9.330001e+14
4,F505,09/04/2018,08:14:20,5.0,0.0,22.0,22.0,7.0,21.0,"[1.0, 8.0, 10.0, 12.0, 15.0, 16.0, 22.0, 0.0, ...","[15.0, 44.0, 142.0, 353.0, 474.0, 738.0, 1033....",N/A,5,PRESHOCK,cocaine,00:00:00,0001-01-01,9.330001e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,Unnamed: 58,0,0,0.0,0.0,0.0,0.0,0.0,13.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",N/A,5,PRESHOCK,cocaine,00:00:00,0001-01-01,NaN
58,Unnamed: 59,0,0,0.0,0.0,0.0,0.0,0.0,12.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",N/A,5,PRESHOCK,cocaine,00:00:00,0001-01-01,NaN
59,Unnamed: 60,0,0,0.0,0.0,0.0,0.0,0.0,30.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",N/A,5,PRESHOCK,cocaine,00:00:00,0001-01-01,NaN
60,Unnamed: 61,0,0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",N/A,5,PRESHOCK,cocaine,00:00:00,0001-01-01,NaN


In [149]:
dff.fillna({'rfid':-999}, inplace=True)
dff = dff[characteristics_SHOCK]
dff

,rfid,subject,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,total_active_lever_presses,total_inactive_lever_presses,total_shocks,total_reward,rewards_after_first_shock,rewards_got_shock,reward_timestamps
0,9.330001e+14,F501,N/A,5,PRESHOCK,cocaine,1.0,08:14:10,00:00:00,09/04/2018,0001-01-01,39.0,2.0,9.0,34.0,33.0,"[1.0, 2.0, 10.0, 12.0, 14.0, 16.0, 22.0, 24.0,...","[2.0, 33.0, 66.0, 106.0, 141.0, 195.0, 262.0, ..."
1,9.330001e+14,F502,N/A,5,PRESHOCK,cocaine,2.0,08:14:13,00:00:00,09/04/2018,0001-01-01,0.0,5.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,9.330001e+14,F503,N/A,5,PRESHOCK,cocaine,3.0,08:14:16,00:00:00,09/04/2018,0001-01-01,97.0,3.0,12.0,38.0,33.0,"[5.0, 6.0, 7.0, 15.0, 16.0, 19.0, 21.0, 23.0, ...","[41.0, 63.0, 83.0, 105.0, 169.0, 195.0, 219.0,..."
3,9.330001e+14,F504,N/A,5,PRESHOCK,cocaine,4.0,08:14:18,00:00:00,09/04/2018,0001-01-01,52.0,7.0,9.0,26.0,24.0,"[2.0, 6.0, 8.0, 13.0, 16.0, 19.0, 24.0, 25.0, ...","[60.0, 138.0, 247.0, 279.0, 305.0, 364.0, 582...."
4,9.330001e+14,F505,N/A,5,PRESHOCK,cocaine,5.0,08:14:20,00:00:00,09/04/2018,0001-01-01,22.0,0.0,7.0,22.0,21.0,"[1.0, 8.0, 10.0, 12.0, 15.0, 16.0, 22.0, 0.0, ...","[15.0, 44.0, 142.0, 353.0, 474.0, 738.0, 1033...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,-9.990000e+02,Unnamed: 58,N/A,5,PRESHOCK,cocaine,0.0,0,00:00:00,0,0001-01-01,0.0,0.0,0.0,0.0,13.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
58,-9.990000e+02,Unnamed: 59,N/A,5,PRESHOCK,cocaine,0.0,0,00:00:00,0,0001-01-01,0.0,0.0,0.0,0.0,12.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
59,-9.990000e+02,Unnamed: 60,N/A,5,PRESHOCK,cocaine,0.0,0,00:00:00,0,0001-01-01,0.0,0.0,0.0,0.0,30.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
60,-9.990000e+02,Unnamed: 61,N/A,5,PRESHOCK,cocaine,0.0,0,00:00:00,0,0001-01-01,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [150]:
dff['start_time'] = dff['start_time'].apply(lambda x: datetime.strptime(x, "%H:%M:%S").time())
dff

TypeError: strptime() argument 1 must be str, not int

In [132]:
dff['start_date'] = dff['start_date'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y").date())
dff

,rfid,subject,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,total_active_lever_presses,total_inactive_lever_presses,total_shocks,total_reward,rewards_after_first_shock,rewards_got_shock,reward_timestamps
0,933000120138578,F301,N/A,3,SHOCK01,cocaine,1,07:33:15,00:00:00,2018-03-15,0001-01-01,34.0,6.0,6.0,21.0,20.0,"[1.0, 8.0, 9.0, 12.0, 13.0, 17.0, 0.0, 0.0, 0....","[357.0, 392.0, 507.0, 603.0, 746.0, 771.0, 900..."
1,933000120138568,F302,N/A,3,SHOCK01,cocaine,2,07:33:19,00:00:00,2018-03-15,0001-01-01,12.0,1.0,3.0,12.0,9.0,"[3.0, 4.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[5.0, 47.0, 280.0, 306.0, 446.0, 997.0, 2327.0..."
2,933000120138400,F303,N/A,3,SHOCK01,cocaine,3,07:33:21,00:00:00,2018-03-15,0001-01-01,30.0,0.0,7.0,22.0,16.0,"[6.0, 7.0, 9.0, 13.0, 14.0, 15.0, 21.0, 0.0, 0...","[1132.0, 1174.0, 1201.0, 1242.0, 1365.0, 1396...."
3,933000120138403,F304,N/A,3,SHOCK01,cocaine,4,07:33:25,00:00:00,2018-03-15,0001-01-01,28.0,0.0,7.0,25.0,23.0,"[2.0, 3.0, 4.0, 11.0, 12.0, 13.0, 24.0, 0.0, 0...","[12.0, 54.0, 136.0, 222.0, 325.0, 486.0, 590.0..."
4,933000120138386,F305,N/A,3,SHOCK01,cocaine,5,07:33:28,00:00:00,2018-03-15,0001-01-01,36.0,1.0,9.0,30.0,29.0,"[1.0, 7.0, 8.0, 18.0, 19.0, 20.0, 22.0, 28.0, ...","[5.0, 32.0, 77.0, 99.0, 123.0, 269.0, 333.0, 3..."
5,933000120138398,F306,N/A,3,SHOCK01,cocaine,6,07:33:30,00:00:00,2018-03-15,0001-01-01,38.0,0.0,9.0,31.0,29.0,"[2.0, 3.0, 6.0, 11.0, 14.0, 15.0, 21.0, 27.0, ...","[1.0, 26.0, 88.0, 158.0, 242.0, 299.0, 573.0, ..."
6,933000120138579,F307,N/A,3,SHOCK01,cocaine,7,07:33:33,00:00:00,2018-03-15,0001-01-01,11.0,0.0,3.0,11.0,7.0,"[4.0, 5.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.0, 31.0, 486.0, 812.0, 1100.0, 1527.0, 1975..."
7,933000120138561,F308,N/A,3,SHOCK01,cocaine,8,07:33:36,00:00:00,2018-03-15,0001-01-01,16.0,0.0,3.0,12.0,10.0,"[2.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[5.0, 75.0, 394.0, 574.0, 775.0, 1123.0, 1867...."
8,933000120138648,F309,N/A,3,SHOCK01,cocaine,1,08:54:04,00:00:00,2018-03-15,0001-01-01,26.0,5.0,6.0,23.0,21.0,"[2.0, 3.0, 10.0, 15.0, 16.0, 19.0, 0.0, 0.0, 0...","[308.0, 345.0, 402.0, 440.0, 506.0, 666.0, 843..."
9,933000120138642,F310,N/A,3,SHOCK01,cocaine,2,08:54:08,00:00:00,2018-03-15,0001-01-01,27.0,0.0,8.0,25.0,21.0,"[4.0, 6.0, 9.0, 11.0, 12.0, 13.0, 21.0, 24.0, ...","[8.0, 35.0, 59.0, 84.0, 107.0, 238.0, 331.0, 5..."


In [86]:
print(pd.to_datetime('08:16:23').time())

08:16:23


In [87]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45 entries, 0 to 44
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   rfid                          45 non-null     int64  
 1   subject                       45 non-null     object 
 2   room                          45 non-null     object 
 3   cohort                        45 non-null     int64  
 4   trial_id                      45 non-null     object 
 5   drug                          45 non-null     object 
 6   box                           45 non-null     int64  
 7   start_time                    45 non-null     object 
 8   end_time                      45 non-null     object 
 9   start_date                    45 non-null     object 
 10  end_date                      45 non-null     object 
 11  total_active_lever_presses    45 non-null     float64
 12  total_inactive_lever_presses  45 non-null     float64
 13  total_s

In [35]:
filename = ws.split('.')[0]+'_transformed.csv'
filename   
dff.to_csv(os.path.join(output_path, filename))

In [89]:
dff

,rfid,subject,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,total_active_lever_presses,total_inactive_lever_presses,total_shocks,total_reward,rewards_after_first_shock,rewards_got_shock,reward_timestamps
0,933000120124701,F1,N/A,1,SHOCK01,cocaine,1,08:16:23,00:00:00,2017-09-06,0001-01-01,31.0,1.0,9.0,29.0,28.0,"[1.0, 2.0, 9.0, 13.0, 16.0, 18.0, 22.0, 23.0, ...","[26.0, 54.0, 92.0, 117.0, 194.0, 276.0, 354.0,..."
1,933000120124703,F2,N/A,1,SHOCK01,cocaine,2,08:16:28,00:00:00,2017-09-06,0001-01-01,42.0,0.0,13.0,41.0,39.0,"[2.0, 7.0, 8.0, 11.0, 16.0, 20.0, 22.0, 26.0, ...","[12.0, 45.0, 76.0, 144.0, 179.0, 243.0, 329.0,..."
2,933000120117342,F3,N/A,1,SHOCK01,cocaine,3,08:16:32,00:00:00,2017-09-06,0001-01-01,1.0,2.0,0.0,1.0,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1844.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
3,933000120117333,F4,N/A,1,SHOCK01,cocaine,4,08:16:35,00:00:00,2017-09-06,0001-01-01,29.0,0.0,9.0,29.0,28.0,"[1.0, 2.0, 8.0, 12.0, 18.0, 19.0, 23.0, 24.0, ...","[3.0, 72.0, 255.0, 372.0, 466.0, 535.0, 698.0,..."
4,933000120117347,F5,N/A,1,SHOCK01,cocaine,5,08:16:38,00:00:00,2017-09-06,0001-01-01,2.0,1.0,0.0,2.0,2.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[637.0, 1848.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,933000120117349,F6,N/A,1,SHOCK01,cocaine,6,08:16:43,00:00:00,2017-09-06,0001-01-01,27.0,0.0,8.0,27.0,19.0,"[8.0, 9.0, 10.0, 11.0, 13.0, 18.0, 21.0, 24.0,...","[1.0, 162.0, 214.0, 245.0, 304.0, 339.0, 645.0..."
6,933000120117340,F7,N/A,1,SHOCK01,cocaine,7,08:16:46,00:00:00,2017-09-06,0001-01-01,22.0,1.0,6.0,19.0,16.0,"[3.0, 8.0, 10.0, 14.0, 16.0, 18.0, 0.0, 0.0, 0...","[25.0, 57.0, 87.0, 112.0, 222.0, 296.0, 327.0,..."
7,933000120117351,F8,N/A,1,SHOCK01,cocaine,8,08:17:02,00:00:00,2017-09-06,0001-01-01,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,933000120124705,M1,N/A,1,SHOCK01,cocaine,9,08:17:09,00:00:00,2017-09-06,0001-01-01,14.0,1.0,4.0,12.0,9.0,"[3.0, 6.0, 7.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[14.0, 56.0, 251.0, 293.0, 664.0, 947.0, 1161...."
9,933000120117338,M2,N/A,1,SHOCK01,cocaine,10,08:17:11,00:00:00,2017-09-06,0001-01-01,31.0,2.0,7.0,25.0,21.0,"[4.0, 5.0, 6.0, 12.0, 16.0, 18.0, 22.0, 0.0, 0...","[197.0, 278.0, 478.0, 912.0, 984.0, 1070.0, 11..."


In [90]:
dff[~dff['subject'].isin(existed)]

,rfid,subject,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,total_active_lever_presses,total_inactive_lever_presses,total_shocks,total_reward,rewards_after_first_shock,rewards_got_shock,reward_timestamps
16,933000120117341,F9,N/A,1,SHOCK01,cocaine,1,09:43:26,00:00:00,2017-09-06,0001-01-01,25.0,15.0,6.0,22.0,19.0,"[3.0, 5.0, 9.0, 13.0, 14.0, 18.0, 0.0, 0.0, 0....","[14.0, 82.0, 111.0, 191.0, 385.0, 822.0, 860.0..."
17,933000120117334,F10,N/A,1,SHOCK01,cocaine,2,09:43:30,00:00:00,2017-09-06,0001-01-01,63.0,222.0,15.0,52.0,51.0,"[1.0, 2.0, 4.0, 17.0, 18.0, 19.0, 22.0, 24.0, ...","[4.0, 33.0, 63.0, 98.0, 127.0, 149.0, 173.0, 2..."
18,933000120117343,F11,N/A,1,SHOCK01,cocaine,3,09:43:32,00:00:00,2017-09-06,0001-01-01,32.0,1.0,7.0,22.0,21.0,"[1.0, 2.0, 4.0, 13.0, 15.0, 17.0, 22.0, 0.0, 0...","[35.0, 56.0, 86.0, 139.0, 306.0, 459.0, 574.0,..."
19,933000120117355,F12,N/A,1,SHOCK01,cocaine,4,09:43:35,00:00:00,2017-09-06,0001-01-01,31.0,0.0,9.0,30.0,29.0,"[1.0, 4.0, 7.0, 13.0, 16.0, 20.0, 27.0, 29.0, ...","[15.0, 40.0, 65.0, 90.0, 116.0, 280.0, 304.0, ..."
20,933000120117346,F13,N/A,1,SHOCK01,cocaine,5,09:43:37,00:00:00,2017-09-06,0001-01-01,26.0,0.0,6.0,21.0,19.0,"[2.0, 3.0, 6.0, 14.0, 18.0, 19.0, 0.0, 0.0, 0....","[18.0, 45.0, 207.0, 471.0, 591.0, 724.0, 1029...."
21,933000120124699,F14,N/A,1,SHOCK01,cocaine,6,09:43:40,00:00:00,2017-09-06,0001-01-01,68.0,1.0,12.0,40.0,39.0,"[1.0, 5.0, 6.0, 12.0, 13.0, 16.0, 22.0, 24.0, ...","[46.0, 80.0, 144.0, 174.0, 200.0, 225.0, 247.0..."
22,933000120124700,F15,N/A,1,SHOCK01,cocaine,7,09:43:45,00:00:00,2017-09-06,0001-01-01,10.0,0.0,2.0,6.0,3.0,"[3.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[103.0, 263.0, 370.0, 411.0, 805.0, 1162.0, 0...."
23,933000120124697,F16,N/A,1,SHOCK01,cocaine,8,09:43:50,00:00:00,2017-09-06,0001-01-01,47.0,1.0,10.0,34.0,30.0,"[4.0, 5.0, 7.0, 15.0, 16.0, 19.0, 21.0, 22.0, ...","[125.0, 155.0, 193.0, 235.0, 261.0, 285.0, 341..."
24,933000120117339,M27,N/A,1,SHOCK01,cocaine,9,09:43:54,00:00:00,2017-09-06,0001-01-01,3.0,2.0,1.0,2.0,0.0,"[2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[13.0, 42.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
25,933000120117350,M28,N/A,1,SHOCK01,cocaine,10,09:43:57,00:00:00,2017-09-06,0001-01-01,19.0,1.0,6.0,17.0,14.0,"[3.0, 7.0, 10.0, 11.0, 13.0, 14.0, 0.0, 0.0, 0...","[82.0, 335.0, 470.0, 560.0, 712.0, 1110.0, 140..."
